<a href="https://colab.research.google.com/github/Mathias0210/Bach/blob/main/Pyth2fors.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import pandas as pd
!pip install mlxtend

In [21]:
from google.colab import files
uploaded = files.upload()
df = pd.read_excel('SalesData.xlsx')
# Vis de første rækker for at validere data
df.head()

Saving SalesDataAnonymiseret.xlsx to SalesDataAnonymiseret.xlsx


,TransaktionsID,RækkeID,KundeID,Navn,Adresse1,Postnr,LevBynavn,Land,ExtSystemId,KategoriID,ProduktID,ProductName,Uom
0,21221405,75277762,996409,Hansens VVS APS,NaN,3230.0,Græsted,DK,lemu-dk,Metro Therm,345166400,Metro Therm el-vandvarmer model 30 type 907 26...,STK
1,21229498,75280255,995782,Krøll Cranes A/S,NaN,3540.0,Lynge,DK,lemu-dk,Slange PA,7830309398,Slange PA6 SW-Ø21 sort 50M,RUL
2,21121214,75280118,995782,Krøll Cranes A/S,NaN,3540.0,Lynge,DK,lemu-dk,C Karakteristik,0422207630,Automatsikring 6KA 3P C 10A 5SY6310-7,STK
3,21072615,75277996,995782,Krøll Cranes A/S,NaN,3540.0,Lynge,DK,lemu-dk,Ligekærv LK,4323400050,"Wiha skruetrækker slimFix LK 3,5 VDE 1000V",STK
4,21072615,75278024,995782,Krøll Cranes A/S,NaN,3540.0,Lynge,DK,lemu-dk,CEE stikpropper 63 ampere,1515134033,CEE stikprop 5 polet 63A 400V IP44,STK


In [22]:
# Tilføjer "_" til produktID
df['ProduktID'] = df['ProduktID'].apply(lambda x: '_'+str(x))

#Sætter alle produktnavne til stort
df['ProductName']=df['ProductName'].str.upper()

In [23]:
# DataFrame til at bygge selve recommenderen
orders = df[['TransaktionsID', 'ProduktID']]
print(orders.shape)
print(orders.head(10))
# DataFrame til at findeproductnavne, bliver benyttet som opslagstabel
products = df[['ProduktID', 'ProductName']].copy()

print(products.shape)
print(products.head(10))

(10000, 2)
   TransaktionsID    ProduktID
0        21221405   _345166400
1        21229498  _7830309398
2        21121214  _0422207630
3        21072615  _4323400050
4        21072615  _1515134033
5        21072615  _1515132035
6        21222356  _4443160332
7        21222356  _4326906029
8        21218223  _2119002052
9        21218223  _2119808667
(10000, 2)
     ProduktID                                        ProductName
0   _345166400  METRO THERM EL-VANDVARMER MODEL 30 TYPE 907 26...
1  _7830309398                         SLANGE PA6 SW-Ø21 SORT 50M
2  _0422207630              AUTOMATSIKRING 6KA 3P C 10A 5SY6310-7
3  _4323400050         WIHA SKRUETRÆKKER SLIMFIX LK 3,5 VDE 1000V
4  _1515134033                 CEE STIKPROP 5 POLET 63A 400V IP44
5  _1515132035             CEE FORLÆNGERLED 5 POLET 63A 400V IP44
6  _4443160332             BOREPATRON TIL M18 FPD/FDD, M18 ONEPD,
7  _4326906029                 TAJIMA 5M HI LOCK 25MM RØD KPS KL1
8  _2119002052  SIKKERHEDSAFBRYDER 16A 3 PO

In [24]:
# Fjerner duplikerede linjer i product dataframe
products = products[~products.duplicated()]


print(products.shape)
print(products.head(10))

(6477, 2)
     ProduktID                                        ProductName
0   _345166400  METRO THERM EL-VANDVARMER MODEL 30 TYPE 907 26...
1  _7830309398                         SLANGE PA6 SW-Ø21 SORT 50M
2  _0422207630              AUTOMATSIKRING 6KA 3P C 10A 5SY6310-7
3  _4323400050         WIHA SKRUETRÆKKER SLIMFIX LK 3,5 VDE 1000V
4  _1515134033                 CEE STIKPROP 5 POLET 63A 400V IP44
5  _1515132035             CEE FORLÆNGERLED 5 POLET 63A 400V IP44
6  _4443160332             BOREPATRON TIL M18 FPD/FDD, M18 ONEPD,
7  _4326906029                 TAJIMA 5M HI LOCK 25MM RØD KPS KL1
8  _2119002052  SIKKERHEDSAFBRYDER 16A 3 POLET  MED HJÆLPEKONT...
9  _2119808667  HJÆLPEKONTAKT 1 SLUTTE FOR 16/125A AFBRYDER OA...


In [25]:
# Fra tidligere trin er der nu kun ca. 6500 linjer i produkt men det bemærkes 

# Downloader produkt dataframen som et led i data preprocessing til at finde hvorfor der er for mange rækker jf. data understanding fasen

#products.to_excel('Products.xlsx')
#files.download('Products.xlsx')

#Det bemærkes at forekllige productnavne har samme produktID, ved nærmere inspektion er det dog de samme produkter bare med forskellige navne.

In [26]:
# Fremviser "fejlen" med duplikerede navne


products = (products
.sort_values(by=['ProduktID'],ascending=False))

products.head(160)

#Se produktID _9978858000	

,ProduktID,ProductName
6964,_RIVS810,"RUSTFRIE STÅLRØR EN 1.4301 GLØDEDE 8X1,00 MM"
5611,_RIVG4832,"RUSTFRIE GEVINDRØR EN 1.4301 GLØDEDE 48,3X3,25 MM"
9581,_RIV4603,"RUSTFRIE STÅLRØR EN 1.4301 60X60X3,00 MM"
858,_RIV4404,"RUSTFRIE STÅLRØR EN 1.4301 40X40X4,00 MM"
8118,_RIV4403,"RUSTFRIE STÅLRØR EN 1.4301 40X40X3,00 MM"
...,...,...
1472,_9978864311,VENTILATOR EDM 100 HZ
9679,_9978863079,SILENT 200 CZ
9044,_9978858000,"THERMEX SILENT ONE, HVID"
3943,_9978858000,THERMEX SILENT ONE Ø100 HVID


In [27]:
#Fjerner fejlene fra Products ved at overskrive dubletterne

products = products.drop_duplicates(subset='ProduktID', keep="last")
print(products.shape)
products.head(160)

(6351, 2)


,ProduktID,ProductName
6964,_RIVS810,"RUSTFRIE STÅLRØR EN 1.4301 GLØDEDE 8X1,00 MM"
5611,_RIVG4832,"RUSTFRIE GEVINDRØR EN 1.4301 GLØDEDE 48,3X3,25 MM"
9581,_RIV4603,"RUSTFRIE STÅLRØR EN 1.4301 60X60X3,00 MM"
858,_RIV4404,"RUSTFRIE STÅLRØR EN 1.4301 40X40X4,00 MM"
8118,_RIV4403,"RUSTFRIE STÅLRØR EN 1.4301 40X40X3,00 MM"
...,...,...
1472,_9978864311,VENTILATOR EDM 100 HZ
9679,_9978863079,SILENT 200 CZ
3943,_9978858000,THERMEX SILENT ONE Ø100 HVID
2629,_9978857386,SILENT 100 CHRZ - DESIGN


In [28]:
# Sætter index til at være kategoriID

products = products.set_index('ProduktID')

# Konverterer til serie for at lave lettere opslag senere
products = products['ProductName']

#Fremviser typen
print(products)
print(type(products))

ProduktID
_RIVS810           RUSTFRIE STÅLRØR EN 1.4301 GLØDEDE 8X1,00 MM
_RIVG4832     RUSTFRIE GEVINDRØR EN 1.4301 GLØDEDE 48,3X3,25 MM
_RIV4603               RUSTFRIE STÅLRØR EN 1.4301 60X60X3,00 MM
_RIV4404               RUSTFRIE STÅLRØR EN 1.4301 40X40X4,00 MM
_RIV4403               RUSTFRIE STÅLRØR EN 1.4301 40X40X3,00 MM
                                    ...                        
_000090126                VINKEL 90° SORT MUFFE/MUFFE 1/2 X 3/8
_000090110                    VINKEL 90° SORT MUFFE/MUFFE 1 1/4
_000090108                        VINKEL 90° SORT MUFFE/MUFFE 1
_000090106                      VINKEL 90° SORT MUFFE/MUFFE 3/4
_000090104                      VINKEL 90° SORT MUFFE/MUFFE 1/2
Name: ProductName, Length: 6351, dtype: object
<class 'pandas.core.series.Series'>


In [29]:
#products.to_excel('Products.xlsx')
#files.download('Products.xlsx')

In [30]:
#Der blev fjernet lidt over 100 fejl med denne metode og jeg må antage at produkterne er de samme og tabellen nu er klar til brug


#Jeg går nu videre til min liste af transaktioner
#Et loop der laver en liste med alle kategoriID'er for hver transaktion og konsoliderer ordrer med alle produkter der har været i

def string_list(x):
    return [str(i) for i in x]

orders = orders.groupby('TransaktionsID')['ProduktID'].apply(list).reset_index()

#Der var sneget sig et par ekstra [] ind i produktname som jeg her fjerner

df['ProduktID'] = df['ProduktID'].str.strip('[]')
orders.head(100)


,TransaktionsID,ProduktID
0,16458869,[_1016008150]
1,16671499,[_3980020020]
2,16686990,"[_4419003986, _4438001708, _728428023]"
3,17632461,[_980419066]
4,18029570,[_7818230573]
...,...,...
95,21051238,[_7897992599]
96,21052330,[_7897700998]
97,21052385,"[_4333956718, _4333956890, _4333956719, _43299..."
98,21053098,"[_4361904104, _5788180022, _RIVS810]"


In [31]:
# Downloader filen som excel for at validere at data er ok

#orders.to_excel('orders.xlsx')
#files.download('orders.xlsx')

orders.shape

(3084, 2)

In [32]:
#Importerer mlxtend https://towardsdatascience.com/mlxtend-a-python-library-with-interesting-tools-for-data-science-tasks-d54c723f89cd

from mlxtend.preprocessing import TransactionEncoder

In [33]:
# Definerer transactionencoder funktionen som en variabel
te = TransactionEncoder()

#Omdanner til en matrix med KategoriID som kolonner og transaktioner som rækker.
te.fit(orders['ProduktID'])
orders_1hot = te.transform(orders['ProduktID'])

# Te.columns giver adgang til de unikke værdier i kategoriID som svarer til 
orders_1hot = pd.DataFrame(orders_1hot, columns =te.columns_)
orders_1hot.head()

,_000090104,_000090106,_000090108,_000090110,_000090126,_000090404,_000090406,_000092104,_000092106,_000092108,...,_RGKF303,_RIG14832,_RIHF3320,_RIV4253,_RIV4303,_RIV4403,_RIV4404,_RIV4603,_RIVG4832,_RIVS810
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [34]:
#Importerer apriori som er vores unsupervised metode
from mlxtend.frequent_patterns import apriori

#Instantierer apriori funktionen med datasettet der er klargjort og viser itemsets ud fra support

ap = apriori(orders_1hot, min_support=0.004, max_len=2, use_colnames=True)
ap = ap.sort_values(by="support",ascending=False)
ap.head(10)

,support,itemsets
5,0.011673,(_1017036338)
20,0.011349,(_1017056279)
24,0.010376,(_1017056402)
16,0.010376,(_1017054433)
19,0.010052,(_1017056266)
46,0.008755,"(_1017056402, _1017056279)"
8,0.008431,(_1017037023)
14,0.008431,(_1017041679)
38,0.008431,(_9221164140)
27,0.007782,(_1017059373)


In [35]:
from mlxtend.frequent_patterns import association_rules

#Viser her de stærkeste associationer
rules = association_rules(ap,metric="lift", min_threshold = 1.2)
print(rules.head(100))

      antecedents    consequents  antecedent support  consequent support  \
0   (_1017056402)  (_1017056279)            0.010376            0.011349   
1   (_1017056279)  (_1017056402)            0.011349            0.010376   
2   (_1017037023)  (_1017037764)            0.008431            0.007782   
3   (_1017037764)  (_1017037023)            0.007782            0.008431   
4   (_1017056279)  (_1017036338)            0.011349            0.011673   
5   (_1017036338)  (_1017056279)            0.011673            0.011349   
6   (_1017056363)  (_1017056266)            0.006809            0.010052   
7   (_1017056266)  (_1017056363)            0.010052            0.006809   
8   (_1017056295)  (_1017056279)            0.007782            0.011349   
9   (_1017056279)  (_1017056295)            0.011349            0.007782   
10  (_1017056402)  (_1017036338)            0.010376            0.011673   
11  (_1017036338)  (_1017056402)            0.011673            0.010376   
12  (_101705

In [36]:
#Funktion der laver foreslag baseret på produktID

def predict(antecedent, rules, max_results= 5):
    
    # Finder reglerne for den gældende antedecent
    preds = rules[rules['antecedents'] == antecedent]
    preds = preds['consequents'].apply(iter).apply(next)
    
    # Returnerer antal forslag som der er angivet i predict funktionen
    return preds[:max_results]

In [37]:
    # Her skal indsættes det ProduktID som der ønskes anbefalinger ud fra
    preds = predict({'_1017056279'}, rules)

    # Vil returnere produktID for de consequents der hører til antedecent
    preds

1     _1017056402
4     _1017036338
9     _1017056295
12    _1017056334
18    _1017056525
Name: consequents, dtype: object

In [38]:
    # Benytter products til at hente produktnavne fremfor intetsigende produktID
    for produktID in preds:  
      print(products[produktID])

LK FUGA SOFT DESIGNRAMME 1½ MODUL, HVID
FUGA STIKK 2P MJ 1½M, HV
LK FUGA TEKNISK MONTERINGSRAMME FOR SOFT, BASE 63, CHOICE & PURE DESIGNRAMMER 2½ MODUL
LK FUGA TEKNISK MONTERINGSRAMME FOR SOFT, BASE 63, CHOICE & PURE DESIGNRAMMER 2X1½ MODUL
LK FUGA SOFT DESIGNRAMME 2½ MODUL, HVID
